Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.


* Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.
```python
# imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'
# tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
# Un patrón que podriamos usar es: 
​patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

# de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 
```



* Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.


In [47]:
import pandas as pd
import numpy as np
import sidetable
import re

In [48]:
df = pd.read_csv('datos/attacks_limpieza_1.csv', index_col = 0)
df.head(1)

,case_number,year,type,country,area,location,activity,age,species_,date,month,fatal_y/n,sex_m_f
0,2018.06.25,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,57,White shark,25-Jun-2018,Jun,N,F


In [49]:
df2 = df.copy()

In [ ]:
# Primero rellenamos los nulos de la columna species con strings para poder hacer busquedas regex y que no nos de error
df2['species_'] = df2['species_'].fillna('NAN')
df2['species_'].unique()

In [51]:
# Vemos si funcionó. Bien, en 'species' no tenemos nulos
df2.isnull().sum()

case_number      0
year             0
type             0
country         10
area            46
location        51
activity        31
age            154
species_         0
date             0
month          181
fatal_y/n       99
sex_m_f          0
dtype: int64

In [ ]:
# Vamos a comprobar la efectividad de nuestro patron regex para los tiburones tigre
# Primero extraemos todas las apariciones de 'tiger'
df2[df2['species_'].str.contains('tiger', case=False)]['species_'].unique()

In [53]:
# Vemos cuantas han sido
len(df2[df2['species_'].str.contains('tiger', case=False)]['species_'].unique())

93

In [54]:
# Las guardamos en una variable
tiger = df2[df2['species_'].str.contains('tiger', case=False)]['species_'].unique()

In [55]:
# Las convertimos a string para aplicarle regex
tiger2 = ','.join(tiger)

In [56]:
patron_tiger = '[Tt]iger|IGER'

In [ ]:
list_tigers = re.findall(patron_tiger, tiger2)
list_tigers

In [58]:
# Coincide con el numero de apariciones que filtramos del dataframe con str.contains
len(list_tigers)

93

In [59]:
# Nuestro patrón va a encontrar 'tiger' en todas las filas que hemos filtrado

In [60]:
# Como nuestro patrón es distinto al que nos decía el enunciado, comprobamos el patrón de white

In [ ]:
# Filtramos y extraemos valores unicos
df2[df2['species_'].str.contains('white', case=False)]['species_'].unique()

In [62]:
# Vemos el numero de filas que lo contienen
len(df2[df2['species_'].str.contains('white', case=False)]['species_'].unique())

219

In [ ]:
# Guardamos las apariciones en una lista
white = list(df2[df2['species_'].str.contains('white', case=False)]['species_'].unique())
white

In [ ]:
# Pasamos a string para aplicarle regex
white2 = ','.join(white)
white2

In [65]:
patron_blanco = r".*[Ww](hite|HITE).*"

In [66]:
# Hacemos la búsqueda en nuestro string 
whites = re.findall(patron_blanco, white2)
len(whites)

1

In [67]:
#¿Qué ha encontrado regex con este patrón?
whites

['hite']

In [68]:
# Este patron no recoge todas las apariciones, pero nos sirve para identificar nuestra fila para clasificarla

In [69]:
# Sin embargo, elegimos usar nuestro patrón comprobado para cada fila.

In [70]:
patron_blanco2 = r"[Ww]hite|HITE"

In [71]:
whites = re.findall(patron_blanco2, white2)
len(whites)

222

In [72]:
# Definimos función
def clasif_especies (fila):
    patron_tiger = r'[Tt]iger|IGER'
    patron_grey = r'[Gg]rey|REY'
    patron_lemon = r'[Ll]emon|EMON'
    patron_bull = r'[Bb]ull|ULL'
    patron_blanco = r"[Ww]hite|HITE"
    
    if re.findall(patron_blanco, fila):
        return fila.replace(fila, 'white shark')
    elif re.findall(patron_tiger, fila):
        return fila.replace(fila, 'tiger shark')
    elif re.findall(patron_grey, fila):
        return fila.replace(fila, 'grey shark')
    elif re.findall(patron_lemon, fila):
        return fila.replace(fila, 'lemon shark')
    elif re.findall(patron_bull, fila):
        return fila.replace(fila, 'bull shark')
    else:
        return fila.replace(fila, 'unspecified')


In [73]:
# Hacemos una copia para probar la funcion de species
df3 = df2.copy()

In [74]:
df3['species_'] = df3['species_'].apply(clasif_especies)

In [75]:
df3['species_'].value_counts()

unspecified    984
white shark    386
tiger shark    157
bull shark     104
grey shark      30
lemon shark     11
Name: species_, dtype: int64

In [76]:
# Hemos comprobado que no había diferencia de resultado con los patrones dados en el enunciado y los nuestros.
# Lo que sí vemos es que el número de filas de white y tiger shark después de la limpieza es mayor al que habíamos 
# encontrado en las comprobaciones. 
# Esto se debe probablemente a que las comprobaciones estaban hechas con los valores únicos, y habría repeticiones.


2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
* Edad en formato string
* Edades separadas por &, or, to, >
* Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
* Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
* Puede que os salga un error similar a este:
>```python
>TypeError: expected string or bytes-like object
>```
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
>```python
>df['nombre_columna'] = df['nombre_columna'].astype(str)
>```

* Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método `explode`.

In [77]:
df3.head(1)

,case_number,year,type,country,area,location,activity,age,species_,date,month,fatal_y/n,sex_m_f
0,2018.06.25,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,57,white shark,25-Jun-2018,Jun,N,F


In [78]:
# Recordamos qué tipos de variables tenemos
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1671
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   case_number  1672 non-null   object
 1   year         1672 non-null   int64 
 2   type         1672 non-null   object
 3   country      1662 non-null   object
 4   area         1626 non-null   object
 5   location     1621 non-null   object
 6   activity     1641 non-null   object
 7   age          1518 non-null   object
 8   species_     1672 non-null   object
 9   date         1672 non-null   object
 10  month        1491 non-null   object
 11  fatal_y/n    1573 non-null   object
 12  sex_m_f      1672 non-null   object
dtypes: int64(1), object(12)
memory usage: 182.9+ KB


In [79]:
# Aplicamos el cambio de dtype
df3['age'] = df3['age'].astype(str)

In [80]:
# No parece haber ningun cambio
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1671
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   case_number  1672 non-null   object
 1   year         1672 non-null   int64 
 2   type         1672 non-null   object
 3   country      1662 non-null   object
 4   area         1626 non-null   object
 5   location     1621 non-null   object
 6   activity     1641 non-null   object
 7   age          1672 non-null   object
 8   species_     1672 non-null   object
 9   date         1672 non-null   object
 10  month        1491 non-null   object
 11  fatal_y/n    1573 non-null   object
 12  sex_m_f      1672 non-null   object
dtypes: int64(1), object(12)
memory usage: 182.9+ KB


In [81]:
# Vemos cuántos valores únicos tenemos y cuáles son antes de la limpieza
df3['age'].unique().size

95

In [82]:
df3['age'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', 'Teen', '30s', '77', '8 or 10',
       '84', '\xa0 ', ' ', '30 or 36', '6½', '5', ' 30', ' 28', "60's",
       '67', '>50', '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18',
       '7 or 8', '9 or 10', 'nan', 'young', '13 or 14'], dtype=object)

In [83]:
# Definimos función
def sacar_age (fila):
    patron1 = '\d+'
    lista = re.findall(patron1, fila)
    if len(lista) > 1:
        for i in range(len(lista)):
            lista[i] = int(lista[i]) 
        return str(int(sum(lista)/len(lista))) # para que nos haga la media de las edades si encuentra más de una. Volvemos a pasar a str para luego hacer un astype general
    elif len(lista) == 1:
        return lista[0]    # para que nos devuelva el elemento encontrado
    else:
        return np.nan   # si la lista está vacía, devolver nan
            

In [84]:
df4 = df3.copy()
df4.head(2)

,case_number,year,type,country,area,location,activity,age,species_,date,month,fatal_y/n,sex_m_f
0,2018.06.25,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,57,white shark,25-Jun-2018,Jun,N,F
1,2018.06.03.a,2018,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,18,tiger shark,03-Jun-2018,Jun,Y,M


In [85]:
# Aplicamos el cambio sobre un dataframe copiado
df4['age'] = df4['age'].apply(sacar_age)

In [86]:
# Comprobamos
df4['age'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', '6', '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '9', '44', '14', '62', '52', '38', '68', '16', '47',
       '63', '70', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', nan, '77', '84', '5', '67'], dtype=object)


* Por último cambiad el tipo de la columna de string a integer.


In [87]:
df4['age'] = df4['age'].astype(np.number)

# No nos deja cambiar a int porque tenemos nans, que son float

In [88]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1671
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   case_number  1672 non-null   object 
 1   year         1672 non-null   int64  
 2   type         1672 non-null   object 
 3   country      1662 non-null   object 
 4   area         1626 non-null   object 
 5   location     1621 non-null   object 
 6   activity     1641 non-null   object 
 7   age          1514 non-null   float64
 8   species_     1672 non-null   object 
 9   date         1672 non-null   object 
 10  month        1491 non-null   object 
 11  fatal_y/n    1573 non-null   object 
 12  sex_m_f      1672 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 182.9+ KB


In [89]:
# ¿Cuántos nulos se nos quedan en contraste con el dataframe sin limpiar?
df4.isnull().sum()

case_number      0
year             0
type             0
country         10
area            46
location        51
activity        31
age            158
species_         0
date             0
month          181
fatal_y/n       99
sex_m_f          0
dtype: int64

In [90]:
df2.isnull().sum()

case_number      0
year             0
type             0
country         10
area            46
location        51
activity        31
age            154
species_         0
date             0
month          181
fatal_y/n       99
sex_m_f          0
dtype: int64

In [91]:
# Se nos quedan sólo dos nulos más


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.


In [92]:
df4.to_csv('datos/attacks_limpieza_2.csv')